In [2]:
# Import necessary libraries
import pandas as pd               # For data manipulation using DataFrames
import numpy as np                # For numerical operations
import matplotlib.pyplot as plt   # For data visualization
import os                         # For operating system-related tasks
import joblib                     # For saving and loading models
import hopsworks                  # For getting access to hopsworks



# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

In [4]:
from feature_pipeline import tesla_fg
from feature_pipeline import news_sentiment_fg

         date  1. open  2. high    3. low  4. close    5. volume ticker
0  2024-05-02   182.86   184.60  176.0200    180.01   89148041.0   TSLA
1  2024-05-01   182.00   185.86  179.0100    179.99   92829719.0   TSLA
2  2024-04-30   186.98   190.95  182.8401    183.28  127031787.0   TSLA
3  2024-04-29   188.42   198.87  184.5400    194.05  243869678.0   TSLA
4  2024-04-26   168.85   172.12  166.3700    168.29  109815725.0   TSLA
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549015
Connected. Call `.close()` to terminate connection gracefully.
Index(['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='object')


Uploading Dataframe: 0.00% |          | Rows 0/3485 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: tesla_stock_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549015/jobs/named/tesla_stock_3_offline_fg_materialization/executions


Uploading Dataframe: 0.00% |          | Rows 0/720 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_sentiment_updated_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549015/jobs/named/news_sentiment_updated_2_offline_fg_materialization/executions


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [6]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549015
Connected. Call `.close()` to terminate connection gracefully.


In [18]:
def create_stocks_feature_view(fs, version):

    # Loading in the feature groups
    tesla_fg = fs.get_feature_group('tesla_stock', version=3)
    news_sentiment_fg = fs.get_feature_group('news_sentiment_updated', version=2)

    # Define the query
    ds_query = tesla_fg.select(['date', 'open', 'ticker'])\
        .join(news_sentiment_fg.select_except(['ticker', 'time', 'amp_url', 'image_url']))

    # Create the feature view
    feature_view = fs.create_feature_view(
        name='tsla_stocks_fv',
        query=ds_query,
        labels=['ticker']
    )

    return feature_view, tesla_fg

In [19]:
try:
    feature_view = fs.get_feature_view("tsla_stocks_fv", version=1)
    tesla_fg = fs.get_feature_group('tesla_stock', version=3)
except:
    feature_view, tesla_fg = create_stocks_feature_view(fs, 1)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/549015/featurestores/544840/featureview). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270046,"usrMsg":"HOPSFS Connector: fklitte_Training_Datasets","errorMsg":"HopsFs Connector not found"}', error code: 270046, error msg: HopsFs Connector not found, user msg: HOPSFS Connector: fklitte_Training_Datasets

In [14]:
#def create_stocks_feature_view(fs, version):

    #Loading in the feature groups
#    tesla_fg = fs.get_feature_group('tesla_stock', version = 3)
#    news_sentiment_fg = fs.get_feature_group('news_sentiment_updated', version = 2)

#    ds_query = tesla_fg.select(['date','open', 'ticker'])\
#        .join(news_sentiment_fg.select_except(['ticker','time', 'amp_url', 'image_url']))
    
#    return (fs.create_tesla_feature_view(
#        name = 'tsla_stocks_fv',
#        query = ds_query,
#        labels=['ticker']
#    ), tesla_fg)

In [15]:
#try:
#    feature_view = fs.get_feature_view("tsla_stocks_fv", version=1)
#    tesla_fg = fs.get_feature_group('tesla_stock', version=3)
#except:
#    feature_view, tesla_fg = create_stocks_feature_view(fs, 1)

AttributeError: 'FeatureStore' object has no attribute 'create_tesla_feature_view'